Notebook for Mobile nets via Tensorflow

1. Import required modules

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # makes tensorflow use gpu instead of cpu
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import fiftyone as fo
import random

2a. Load Mobile net model with specified weights

In [29]:
# model = MobileNet(alpha=1,depth_multiplier=10,dropout=0.001,weights=None,classifier_activation='softmax')
model = MobileNet(alpha=1,depth_multiplier=1,dropout=0.001,weights='imagenet',classifier_activation='softmax')
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

2b . If you want: compile the model to train in a specific way

In [30]:
# Compile the model
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

3-Option 1. ONLY DO THIS IF YOU WANT TO FEED IT ONE OF YOUR PICTURES: Then skip to part 6

In [39]:
# Load an image from laptop for classification
img_path = r"C:\Users\Windows\Desktop\im3.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

3-Option 2. Load COCO dataset, using just a portion of it

In [31]:
# From the 80 categories that COCO has, selct two of them to pool images from
categories = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis','snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
category1 = random.choice(categories)
category2 = random.choice(categories)
if category1 == category2:
    category2 = random.choice(categories)
# Display what categories will pictures come from
print('Class 1:',category1,'Class 2:',category2,)

#Load data set using categories and fiftyone module
dataset = fo.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections", "segmentations"],
    classes=[category1, category2],
    max_samples=50,
)

Class 1: cell phone Class 2: umbrella
Found annotations at 'C:\Users\Windows\fiftyone\coco-2017\raw\instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation-50'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


 4. Select picture to predict

In [32]:
# Randomly select an image from the dataset
sample = dataset.take(1).first()

In [34]:
# Visualize the dataset in the FiftyOne App - Not needed
session = fo.launch_app(dataset)

5. Load image for classification

In [35]:
# look into why image is converted into an array
img = image.load_img(sample.filepath, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

6. Predict class probabilities and decode prediction

In [40]:
# Predict class probabilities
predictions = model.predict(img_array)
# Decode predictions
decoded_predictions = decode_predictions(predictions, top=3)[0]
# used to retrieve probailities in next section

1/1 [==============================] - 0s 98ms/step


7. display top predictions 

In [41]:
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({score:.2f})")

1: sports_car (0.98)
2: racer (0.01)
3: car_wheel (0.00)


8. Display image

In [42]:
img.show()